In [ ]:
import os
import json
import fitz  # PyMuPDF
from fastapi import FastAPI, UploadFile, File, HTTPException
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
app = FastAPI()

# Setup OpenRouter Client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

SEED_QUESTIONS = [
    "Tell me a little about yourself and what motivates you to wake up in the morning.",
    "Tell me about a time you made a significant mistake at work. How did you handle it?",
    "Describe a situation where you had to deal with a difficult coworker or client.",
    "Have you ever been assigned a task you felt was impossible? What did you do?",
    "If we asked your previous manager to describe you in three words, what would they be and why?",
    "Describe your ideal work environment.",
    "Tell me about a time you had to deliver bad news to a stakeholder.",
    "What is the one professional achievement you are most proud of?",
    "Tell me about a time you had to learn a completely new tool or skill very quickly.",
    "Is there anything about this job description that makes you nervous?"
]

@app.post("/generate-questions")
async def generate_questions(file: UploadFile = File(...)):
    # 1. Extract PDF Text
    pdf_content = await file.read()
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    cv_text = "".join([page.get_text() for page in doc])

    # 2. Build the Prompt
    prompt = f"CV TEXT:\n{cv_text}\n\nSEED QUESTIONS:\n{json.dumps(SEED_QUESTIONS)}\n\n" \
             f"Generate 10 tailored questions based on this CV in JSON format: " \
             f"{{'questions': [{{'id': 1, 'text': '...'}}]}}"

    # 3. Call OpenRouter (Using a free Llama 3.3 model)
    try:
        response = client.chat.completions.create(
            model="meta-llama/llama-3.3-70b-instruct:free",
            messages=[
                {"role": "system", "content": "You are a recruitment assistant. Output ONLY valid JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={ "type": "json_object" } # Forces JSON output
        )
        
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

ModuleNotFoundError: No module named 'fitz'